<a href="https://colab.research.google.com/github/rieldata1/deep-clustering-rails/blob/main/Deep_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Configurar GPU y activar high-RAM**

In [1]:
# Montar Drive
from google import colab
colab.drive.mount('/content/drive')

# Configurar GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# Activar high-RAM
import psutil
ram_gb = psutil.virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Mounted at /content/drive
Tue Sep  2 19:47:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   50C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+---------------------

# **Deep_Clustering: carga NPZ + AE + K-sweep + DEC (+ IDEC)**
* Este notebook consumirá los NPZ generados por "Scalograms".
* Optimizaciones: GPU, AMP (mixed precision), prefetch, pin_memory.


## **0. Preeliminares**

In [4]:
# -----------------------------
# PARÁMETROS (ajustar a voluntad)
# -----------------------------
# Rutas (apunta al índice creado por "Scalograms.ipynb")
BASE_DIR           = "/content/drive/MyDrive/Deep_Cluster"
INDEX_CSV          = f"{BASE_DIR}/meta/scalos_train_index.csv"     # índice principal
EXPERIMENTS_DIR    = f"{BASE_DIR}/experiments"                     # carpeta para guardar resultados (modelos, history)

# Carga de datos
PRELOAD_DATA       = True      # True: intenta precargar todos los NPZ en RAM si caben; False: lee por streaming
PRELOAD_MAX_GB     = 4.0       # umbral aprox para decidir si precarga (ajústalo según tu Colab Pro)
IMG_NORMALIZE_AGAIN= False     # renormalizar a [0,1] al vuelo (no debería hacer falta si Scalograms ya normalizó)

# Sampler (diagnóstico): balancear por etiqueta simulada (si existe en NPZ)
BALANCE_BY_LABEL   = False

# Batching / DataLoader
BATCH_AE           = 64        # batch para preentrenamiento del Autoencoder
BATCH_EMB          = 128       # batch para extraer embeddings
BATCH_DEC          = 64        # batch para DEC/IDEC
NUM_WORKERS        = 2         # sube si tienes CPU libre; si da problemas, pon 0
PIN_MEMORY         = True
PERSISTENT_WORKERS = True

# Modelo
IMG_SIZE           = (256, 256) # se verificará contra el CSV; aquí sirve como referencia
LATENT_DIM         = 128        # tamaño del embedding z
BACKBONE           = "base"     # 'small' | 'base' | 'large' (tamaño de la CNN)
DROPOUT_P          = 0.0

# Entrenamiento AE (preentrenamiento)
AE_EPOCHS          = 25
AE_LR              = 1e-3
AE_WD              = 1e-5
AE_LOSS            = "l1"       # 'l1' o 'mse'
USE_AMP            = True       # usar mixed precision en GPU (acelera / ahorra RAM)

# Barrido de K (auto-K)
K_MIN              = 2
K_MAX              = 10
K_FIXED            = None       # si quieres saltar el barrido, pon un entero (p. ej., 4)
K_RANDOM_STATE     = 2025

# DEC (clustering profundo)
DEC_EPOCHS         = 40
DEC_LR             = 1e-4
DEC_WD             = 0.0
DEC_UPDATE_INT     = 1          # actualizar distribución objetivo P cada N épocas
DEC_TOL            = 1e-3       # criterio de estabilidad de asignaciones

# IDEC (opcional: DEC + pérdida de reconstrucción)
RUN_IDEC           = False
IDEC_EPOCHS        = 40
IDEC_LR            = 1e-4
IDEC_WD            = 0.0
IDEC_LAMBDA_REC    = 1e-2
IDEC_UPDATE_INT    = 1
IDEC_TOL           = 1e-3

# Visualización / Evaluación
RUN_TSNE_2D        = True
TSNE_PERPLEXITY    = 30
SEED               = 153

# Guardado de artefactos (pesos, centros, history, config)
SAVE_ARTIFACTS     = True


# -----------------------------
# IMPORTS (solo librerías estándar de Colab)
# -----------------------------
import os, time, csv, math, gc, json, random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.manifold import TSNE

from datetime import datetime
from collections import defaultdict, Counter


# -----------------------------
# Runtime / Device (GPU + High-RAM)
# -----------------------------
def seed_everything(seed=SEED):
    """Fija semillas y activa pequeñas optimizaciones del backend."""
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # Para CNNs suele acelerar (usa kernel heurístico por tamaño)
    torch.backends.cudnn.benchmark = True
    # Precisión alta para matmul en GPUs modernas
    if hasattr(torch, "set_float32_matmul_precision"):
        torch.set_float32_matmul_precision("high")

# Asegúrate de seleccionar GPU y High-RAM en: Runtime → Change runtime type
seed_everything(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if DEVICE.type == "cuda":
    print("✅ GPU detectada:", torch.cuda.get_device_name(0))
else:
    print("⚠️  No se detectó GPU: ejecutará en CPU (más lento).")

# Crear carpetas de experimento
os.makedirs(EXPERIMENTS_DIR, exist_ok=True)
RUN_DIR = os.path.join(EXPERIMENTS_DIR, datetime.now().strftime("run_%Y%m%d_%H%M%S"))
os.makedirs(RUN_DIR, exist_ok=True)
print("Carpeta de ejecución:", RUN_DIR)


# -----------------------------
# Utils simples
# -----------------------------
def sizeof_gb(n_items, h, w, bytes_per=2):
    """Estimación de tamaño (GB) para n_items imágenes HxW en float16 (2 bytes por valor)."""
    return (n_items * h * w * bytes_per) / (1024**3)

def count_params(m):
    """Cuenta parámetros entrenables de un modelo."""
    return sum(p.numel() for p in m.parameters() if p.requires_grad)


✅ GPU detectada: NVIDIA L4
Carpeta de ejecución: /content/drive/MyDrive/Deep_Cluster/experiments/run_20250902_200330
